In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

# returns a list containing data split into 80-20 training and testing sets 
def split(df):
    training_set = df.values.tolist()
    
    for x in range(len(training_set)):
        training_set[x].append(0) #add space for euclidean distances
        training_set[x].append("prediction") #add space for prediction labels
    
    test_set = []
    train_test = []
    
    test_size = int(len(training_set) * 0.2)
    for y in range(test_size):
        test_set.append(training_set.pop(random.randint(0, len(training_set) - 1)))
    
    for z in range(len(training_set)):
        training_set[z][6] = training_set[z][4] #set prediction label equal to actual label for training set
        
    train_test.append(training_set)
    train_test.append(test_set)
                    
    return train_test

# computes euclidean distance in terms of sepal length and width given flowers p and q
def get_euclidean_distance(p, q):
    distance = math.sqrt((p[2] - q[2]) ** 2 + (p[3] - q[3]) ** 2)
    return distance

# returns a list containing the 5 nearest neighbours of p
def get_nearest_neighbours(training_set, p):
    nn = []
    
    for x in range(len(training_set)):
        training_set[x][5] = get_euclidean_distance(training_set[x], p)
        nn.append(training_set[x])
        
    nn.sort(key = lambda y: y[5])
    return nn[:6]

# classifies flower p based on its 5 nearest neighbours
def classify(training_set, nn, p):
    labels = ["setosa", "versicolor", "virginica"]
    label_count = [0, 0, 0]
    
    for x in range(5):
        for y in nn[x]:
            if (y in labels):
                label_count[labels.index(y)] += 1
           
    p[6] = labels[label_count.index(max(label_count))]
    
    training_set.append(p)
    
def knn(train_test):    
    training_set = train_test[0]
    test_set = train_test[1]

    for x in range(len(test_set)):
        nn = get_nearest_neighbours(training_set, test_set[x])
        classify(training_set, nn, test_set[x])
        
    for y in range(len(training_set)):
        training_set[y].pop(5)
        
    return training_set, test_set

def get_accuracy(test_set):
    correct_predictions = 0.0
    for x in range(len(test_set)):
        if (test_set[x][4] == test_set[x][5]):
            correct_predictions += 1
    
    result = correct_predictions / len(test_set)
    return (result)

In [51]:
iris_df = pd.read_csv("iris.csv")
train_test = split(iris_df)
results = knn(train_test)

iris_all = results[0]
iris_all_df = pd.DataFrame(iris_all, columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", 
                                           "species", "prediction"])

predictions = results[1]
predictions_df = pd.DataFrame(predictions, columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", 
                                           "species", "prediction"])
print(get_accuracy(predictions))

1.0


In [ ]:
# scatterplots of training and testing sets
actual_scatter = sns.relplot(x = "petal_length", y = "petal_width", hue = "species", data = iris_all_df)
actual_scatter.set(title = "Petal Length and Width of Irises - Expected", 
                 xlabel = "Petal Length (cm)",  
                 ylabel = "Petal Width (cm)")
plt.show()

prediction_scatter = sns.relplot(x = "petal_length", y = "petal_width", hue = "prediction", data = iris_all_df)
prediction_scatter.set(title = "Petal Length and Width of Irises - Actual", 
                 xlabel = "Petal Length (cm)",  
                 ylabel = "Petal Width (cm)")
plt.show()

In [ ]:
# scatterplots of predictions (testing set) only 
actual_scatter = sns.relplot(x = "petal_length", y = "petal_width", hue = "species", data = predictions_df)
actual_scatter.set(title = "Petal Length and Width of Irises - Expected", 
                 xlabel = "Petal Length (cm)",  
                 ylabel = "Petal Width (cm)")
plt.show()

prediction_scatter = sns.relplot(x = "petal_length", y = "petal_width", hue = "prediction", data = predictions_df)
prediction_scatter.set(title = "Petal Length and Width of Irises - Actual", 
                 xlabel = "Petal Length (cm)",  
                 ylabel = "Petal Width (cm)")
plt.show()